# Protocol for analysis of labeled proteomics data
We recommend the following structure for describing the protocol and its example workflow. Below you can find an overview of required features of the protocol and its description and rules for best practices.

This is the link to the github repository of the protocol 
TODO add version!
https://github.com/ProtProtocols/biocontainer-jupyter

Link to docker image:



## Abstract
Provide a short description of the software protocol including broader context, functionality, use case and purpose. 


## Maintainer
Provide details about the protocol maintainer (e.g. email address and/or github username)

## Software
Specify links for documentation and tutorials of used software, source code, publications and use cases. Detail versions of each used software. Alternatively, provide links to the software descriptions in https://bio.tools where this information is available.

## Diagram
Provide a simple diagram of functionality of the workflow/software. We recommend using controlled vocabularies for input/output data types and file formats as well as provided operation of the tool(s). You can use http://edamontology.org terms for the description.

__TODO: example__

## System requirements
Fill in the following items:
Required hard disk space for docker image, input and output files: 

Required memory: 

Recommmended number of threads: 

## Example 
Presentation of well-documented instructions and commands to run the example use case. Depending on the use case and the software, provide link(s) to open the web service incorportated in the Docker image (e.g. 0.0.0.0:8080), bash commands to run programs from the command line and additional code for e.g. checking and visualizing the (intermediate) results. 

Instead of providing the instructions in this notebook, one can also provide a link to a notebook containing the example use case.

## More general use case (optional)
Provide link to notebook with a generalized use case that easily can be adapted to e.g. process different input data and concurrent parametrization.




# Example


Specify parameters for database search and evaluation of identified peptide-spectrum matches:

In [ ]:
%load_ext rpy2.ipython

In [ ]:
import ipywidgets as widgets
from ipywidgets import VBox, Label
import sys, os
import subprocess

# create an empty class as a storage for all UI widgets
class SearchUI:
    def __init__(self):
        self.work_dir_select = widgets.Dropdown(options={'/data/': '/data/', 'Example files': '/home/biodocker/'}, 
                                                value='/home/biodocker/')
        self.work_dir_select.observe(observe_work_dir_select)
        
        self.precursor_tolerance = widgets.IntSlider(min=-10,max=30,step=1,value=20)
        self.fragment_tolerance = widgets.BoundedFloatText(min=0,max=200,value=0.05)
        # TODO: change this to a select box listing all FASTA files in the working directory
        self.fasta_db = widgets.Dropdown(options={"sp_human.fasta": "IN/sp_human.fasta"})
        
        # TODO  needs table to describe labeling formats
        self.labelling = widgets.Dropdown(options=
                          {'TMT6': 'TMT 6-plex of K,TMT 6-plex of peptide N-term',
                           'TMT10': 'TMT 10-plex of K,TMT 10-plex of peptide N-term','iTRAQ4': 'iTRAQ 4-plex of K,iTRAQ 4-plex of Y,iTRAQ 4-plex of peptide N-term',
                           'iTRAQ8 (fixed)': 'iTRAQ 8-plex of K, iTRAQ 8-plex of peptide N-term',
                           'iTRAQ8 (variable)': 'iTRAQ 8-plex of Y'},
                      value='TMT 10-plex of K,TMT 10-plex of peptide N-term')
        
        self.missed_cleavages = widgets.IntSlider(min=0,max=10,step=1,value=1)
        self.fixed_ptms = widgets.Dropdown(options=["Carbamidomethylation of C","None"])

        # PTMs
        self.var_ptms = widgets.SelectMultiple(
            options=["Oxidation of M",
                     "Phosphorylation of STY",
                     "Acetylation of peptide N-term",
                     "Acetylation of protein N-term"],
            value=['Oxidation of M'])
        
        self.spectra_dir = widgets.Dropdown(options={"IN": "IN"})

        # ww = widgets.Checkbox(description="Decoy")

        self.search_button = widgets.Button(
            description='Run Search',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Launch the search',
            icon='check'
        )

        self.search_button.on_click(run_search)
        
        
    def updateFastaFiles(self, workdir):
        # get all FASTA files
        fasta_files = [file for file in os.listdir(workdir) if file[-6:] == ".fasta"]
        
        # also search all subdirectories for FASTA files
        for d in os.listdir(workdir):
            d_path = os.path.join(workdir, d)
            if os.path.isdir(d_path) and d[0] != ".":
                fasta_files += [os.path.join(d, file) for file in os.listdir(d_path) if file[-6:] == ".fasta"]
        
        # create the dict to add as values to the control
        file_list = dict()
        
        for f in fasta_files:
            file_list[f] = os.path.join(os.path.abspath(workdir), f)
        
        self.fasta_db.options = file_list
        
        # update the list of possible peaklist directories
        directories = [d for d in os.listdir(workdir) if os.path.isdir(os.path.join(workdir, d)) and d[0] != "."]
        
        dir_list = dict()
        
        for d in directories:
            dir_list[d] = os.path.join(os.path.abspath(workdir), d)
            
        self.spectra_dir.options = dir_list
        
        if "IN" in dir_list:
            self.spectra_dir.value = dir_list["IN"]
        
        
    def display(self):
        self.updateFastaFiles(self.work_dir_select.value)
        
        settings_box = VBox([Label('Working directory'), self.work_dir_select,
                             Label('Precursor tolerance (ppm):'), self.precursor_tolerance, 
                             Label('Fragment ion tolerance (da):'), self.fragment_tolerance,
                             Label('Fasta file (database, must NOT contain decoy sequences):'), self.fasta_db,
                             Label('Quantification method:'), self.labelling,
                             Label('Number of miscleavages;'), self.missed_cleavages,
                             Label('Further fixed modifications'), self.fixed_ptms,
                             Label('Further variable modifications (Hold Ctrl to select multiple)'), self.var_ptms,
                             Label('Folder for spectra files (files need to be mgf)'), self.spectra_dir,
                             self.search_button])

        display(settings_box)
        

def run_search(button):
    global searchUI, result_file
    
    # make sure all required fields were selected
    if searchUI.work_dir_select.value is None:
        print("Error: No working directory selected")
        return
    
    if searchUI.fasta_db.value is None:
        print("Error: No FASTA file selected")
        return
    
    # create the directory paths to work in
    peaklist_dir = os.path.abspath(searchUI.spectra_dir.value)
    
    if not os.path.isdir(peaklist_dir):
        raise Exception("Invalid peak list directory selected: " + peaklist_dir + " does not exist.")
    
    peptide_shaker_jar = "/home/biodocker/bin/PeptideShaker-1.10.3/PeptideShaker-1.10.3.jar"
    work_dir = os.path.abspath(os.path.join(searchUI.work_dir_select.value, "OUT"))
    
    # the searches should be performed in the "OUT" directory
    if not os.path.isdir(work_dir):
        os.mkdir(work_dir)
        
    print("Creating decoy database...")
    
    # create the decoy database
    subprocess.run(["java", "-cp", "/home/biodocker/bin/SearchGUI-2.8.6/SearchGUI-2.8.6.jar", 
                   "eu.isas.searchgui.cmd.FastaCLI", "-in", searchUI.fasta_db.value, "-decoy"], check=True,
                   cwd=work_dir,
                   stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    # get the filename of the decoy database
    database_file = searchUI.fasta_db.value[:-6] + "_concatenated_target_decoy.fasta"
    
    if not os.path.isfile(os.path.join("OUT", database_file)):
        raise Exception("Failed to find generated decoy datbase")
        
    # build the arguments to create the parameter file
    param_file = os.path.join(work_dir, "search.par")
    
    # remove any old parameters
    if os.path.isfile(param_file):
        os.remove(param_file)
    
    search_args = ["java", "-cp", "/home/biodocker/bin/SearchGUI-2.8.6/SearchGUI-2.8.6.jar",
                   "eu.isas.searchgui.cmd.IdentificationParametersCLI",
                   "-out", param_file]
    
    # precursor tolerance
    search_args.append("-prec_tol")
    search_args.append(str(searchUI.precursor_tolerance.value))
    # fragment tolerance
    search_args.append("-frag_tol")
    search_args.append(str(searchUI.fragment_tolerance.value))
    # fixed mods
    # TODO: labelling cannot always be set as fixed mod???
    fixed_mod_string = str(searchUI.labelling.value) + "," + str(searchUI.fixed_ptms.value)
    search_args.append("-fixed_mods")
    search_args.append(fixed_mod_string)
    # database
    search_args.append("-db")
    search_args.append(database_file)
    # missed cleavages
    search_args.append("-mc")
    search_args.append(str(searchUI.missed_cleavages.value))
    
    # var mods
    if len(searchUI.var_ptms.value) > 0:
        search_args.append("-variable_mods")
        var_mod_list = list()
        
        for var_mod in searchUI.var_ptms.value:
            if var_mod == "Phosphorylation of STY":
                var_mod_list += ["Phosphorylation of S", "Phosphorylation of T", "Phosphorylation of Y"]
            else:
                var_mod_list.append(var_mod)
                
        search_args.append(",".join(var_mod_list))
        
    # create the search parameter file
    print("Creating search parameter file...")
    # print(" ".join(search_args))
    subprocess.run(search_args, check=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if not os.path.isfile(param_file):
        raise Exception("Failed to create search parameters")
        
    # run the search
    print("Running search...")
    # TODO: create list of spectrum files - or the folder
    spectrum_files = peaklist_dir
    print("  Searching files in " + spectrum_files)
    subprocess.run(["java", "-cp", "/home/biodocker/bin/SearchGUI-2.8.6/SearchGUI-2.8.6.jar",
                    "eu.isas.searchgui.cmd.SearchCLI", "-spectrum_files", spectrum_files,
                    "-output_folder", work_dir, "-id_params", param_file,
                    "-xtandem", "0", "-msgf", "1", "-comet", "0", "-ms_amanda", "0", 
                    "-myrimatch", "0", "-andromeda", "0", "-omssa", "0", "-tide", "0"],
                    check=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    print("Search completed.")
    
    # run PeptideShaker
    print("Processing result using PeptideShaker...")
    peptide_shaker_result_file = os.path.join(work_dir, "experiment.cpsx")

    subprocess.run(["java", "-cp", peptide_shaker_jar,
                    "eu.isas.peptideshaker.cmd.PeptideShakerCLI",
                    "-experiment", "experiment1",
                    "-sample", "test",
                    "-replicate", "1",
                    "-identification_files", work_dir,
                    "-out", peptide_shaker_result_file,
                    "-id_params", param_file,
                    "-spectrum_files", spectrum_files],
                    check=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if not os.path.isfile(peptide_shaker_result_file):
        raise Exception("Failed to process result file.")
        
    # create TSV output files
    print("Converting result to TSV format...")
    subprocess.run(["java", "-cp", peptide_shaker_jar,
                  "eu.isas.peptideshaker.cmd.ReportCLI",
                  "-in", peptide_shaker_result_file,
                  "-out_reports", work_dir,
                  "-reports", "8"],
                  check=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    result_file=os.path.join(work_dir, "experiment1_test_1_Extended_PSM_Report.txt")
    
    if not os.path.isfile(result_file):
        raise Exception("Error: Conversion failed")
        
    print("Done.")
    
    
def observe_work_dir_select(change):
    global work_dir_select
    if change['type'] == "change" and change['name'] == "value":
        # update the required UI controls
        searchUI.updateFastaFiles(change["new"])

# -------------------
# Code to create the UI
# --------------------
result_file=None
work_dir=None
peaklist_dir=os.path.abspath("IN")

searchUI = SearchUI()
searchUI.display()

#TODO set names of samples and replicates (peptideshaker)


In [ ]:
%%R -i result_file,peaklist_dir

# library causes the execution to fail if the library is missing
suppressWarnings(suppressMessages(library(isobar)))

# process the input files
max.fdr <- 0.01
# TODO: set based on python selection
quant.method <- "TMT10plexSpectra"
class.labels <- c("A", "B", "C", "D", "E", "F", "G", "H", "I", "J")
args <- commandArgs(trailingOnly = TRUE)

ident.file <- result_file
mgf.files <- list.files(peaklist_dir, pattern="*.mgf$", full.name=T)

if (!file.exists(ident.file)) {
    stop("Error: Cannot find identification file ", ident.file)
}
if (length(mgf.files) < 0) {
    stop("Error: No MGF files found.")
}
for (mgf.file in mgf.files) {
    if (!file.exists(mgf.file)) {
        stop("Error: Cannot find MGF file ", mgf.file)
    }
}

# Convert SearchGUI output to isobar format
psms <- read.csv(ident.file, sep = "\t")

if (! "Decoy" %in% names(psms)) {
    stop("Error: No decoy information available in output file")
}

print(paste("Loaded",nrow(psms), "PSMs"))

# ---- Confidence filter ----
psms <- psms[order(psms[, "Confidence...."], decreasing = T), ]
decoy.psms <- which(psms[, "Decoy"] == "1")

decoy.count <- 0

for (decoy.index in decoy.psms) {
    decoy.count <- decoy.count + 1
    target.count <- decoy.index - decoy.count

    cur.fdr <- (decoy.count * 2) / (decoy.count + target.count)

    if (cur.fdr > max.fdr) {
        # filter
        psms <- psms[1:decoy.index - 1,]
        break
    }
}

# print(head(psms))

print(paste0("Filtered ", nrow(psms), " PSMs @ ", max.fdr, " FDR"))

# ---- convert to isobar output ----
cols.to.save <- c("Protein.s.", "Sequence", "Spectrum.Title", "Variable.Modifications", "Confidence....", "D.score", "Validation", "Precursor.m.z.Error..ppm.", "Spectrum.File")

if (!all(cols.to.save %in% colnames(psms))) {
    stop("Error: Unexpected result format")
}

psms <- psms[, cols.to.save]
colnames(psms) <- c("accession", "peptide", "spectrum", "var_mod", "pepscore", "dscore", "validation", 
"precursor.mz.error.ppm", "file")

# TODO: add modif...
psms$modif <- ""

write.table(psms, file = "t.corr.csv", sep = "\t", row.names = F, quote = F)

# ---- isobar workflow ----
ib <- readIBSpectra(quant.method, "t.corr.csv", mgf.files, decode.titles = T)
ib <- correctIsotopeImpurities(ib)
ib <- normalize(ib)

# Extract peptide and protein ratios
noise.model <- NoiseModel(ib)
suppressWarnings(protein.ratios <- proteinRatios(ib, noise.model, combn.method = "versus.channel", cl = class.labels))
suppressWarnings(peptide.ratios <- peptideRatios(ib, noise.model, combn.method = "versus.channel", cl = class.labels))

boxplot(peptide.ratios$lratio, main = "Peptide ratios", ylab = "Peptide ratio")
boxplot(protein.ratios$lratio, main = "Protein ratios", ylab = "Protein reatios")

# save the ratios for later use
saveRDS(protein.ratios, file = "OUT/protein.ratios.rds")
saveRDS(peptide.ratios, file = "OUT/peptide.ratios.rds")